In [ ]:
import os
import io
import boto3
import pandas as pd
import pyarrow.parquet as pq
%matplotlib widget

## Download a Parquet file from S3
See https://kafka-connect-manager.lsst.io/ for more information on the S3 Sink connector.

In [ ]:
BUCKET_NAME = "efd-sandbox.data"

The S3 credentials can be added to `~/.aws/credentials` file. They are stored in SQuaRE 1Password. Search for EFD AWS S3 credentials. The S3 region can be added to the`~/.aws/config` file.

For example:
```
cat ~/.aws/credentials
[default]
aws_access_key_id = <the aws_access_key_id>
aws_secret_access_key = <the aws_secret_access_key>
```
and
```
cat ~/.aws/config 
[default]
region=us-east-1
```

In [ ]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(BUCKET_NAME)

In this example the S3 Sink connector is configured to partition data by time on an hourly basis. The following helps to construct the path to find the Parquet files on S3.

In [ ]:
topic = "example-002-aggregated"
year = "2020"
month = "08"
day = "07"
hour = "22"

We use the `bucket.download_fileobj()` method to download the Parquet files into a buffer, and then Pyarrow to read the files and convert and append them to a Pandas dataframe.

In [ ]:
df = pd.DataFrame()
for obj in bucket.objects.filter(Prefix=f"topics/{topic}/year={year}/month={month}/day={day}/hour={hour}"):
    buffer = io.BytesIO()
    bucket.download_fileobj(obj.key, buffer)
    df = df.append(pq.read_table(buffer).to_pandas())
    print(f"{bucket.name}:{obj.key}")

The S3 Sink connector is configured to invoke file commits to S3 every 10 minutes (see the `rotate_interval_ms` configuration setting) that's why you see 6 files in this path.

In [ ]:
df.head()

## Plotting the aggregated stream

In [ ]:
p = df.plot(x='time', y='mean_value1', c='white', figsize=(15,5))
p.fill_between(x='time', y1='min_value1', y2='max_value1', data=df)